# CIC IDS 2017

# 1. Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from prettytable import PrettyTable

In [2]:
for dirname, _, filenames in os.walk('data/CIC_IDS_2017/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/CIC_IDS_2017/README.md
data/CIC_IDS_2017/MachineLearningCSV.zip
data/CIC_IDS_2017/GeneratedLabelledFlows.md5
data/CIC_IDS_2017/GeneratedLabelledFlows.zip
data/CIC_IDS_2017/download_dataset
data/CIC_IDS_2017/MachineLearningCSV.md5
data/CIC_IDS_2017/TrafficLabelling /Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Monday-WorkingHours.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Friday-WorkingHours-Morning.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Tuesday-WorkingHours.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv
data/CIC_IDS_2017/TrafficLabelling /Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
data/CIC_IDS_2017/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
data/CIC_IDS_20

# 2. Load data

In [27]:
monday = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv', sep=",", encoding='utf-8')
tuesday = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv', sep=",", encoding='utf-8')
wednesday = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv', sep=",", encoding='utf-8')
thursday_morning = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', sep=",", encoding='utf-8')
thursday_afternoon = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', sep=",", encoding='utf-8')
friday_ddos = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', sep=",", encoding='utf-8')
friday_pcap = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', sep=",", encoding='utf-8')
friday_morning = pd.read_csv('data/CIC_IDS_2017/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv', sep=",", encoding='utf-8')

In [28]:
monday.name = 'Monday'
tuesday.name = 'Tuesday'
wednesday.name = 'Wednesday'
thursday_morning.name = 'Thursday_Morning'
thursday_afternoon.name = 'Thursday_Afternoon'
friday_ddos.name = 'Friday_DDoS'
friday_pcap.name = 'Friday_Pcap'
friday_morning.name = 'Friday_Morning'

In [4]:
monday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529918 entries, 0 to 529917
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             529918 non-null  int64  
 1    Flow Duration                529918 non-null  int64  
 2    Total Fwd Packets            529918 non-null  int64  
 3    Total Backward Packets       529918 non-null  int64  
 4   Total Length of Fwd Packets   529918 non-null  int64  
 5    Total Length of Bwd Packets  529918 non-null  int64  
 6    Fwd Packet Length Max        529918 non-null  int64  
 7    Fwd Packet Length Min        529918 non-null  int64  
 8    Fwd Packet Length Mean       529918 non-null  float64
 9    Fwd Packet Length Std        529918 non-null  float64
 10  Bwd Packet Length Max         529918 non-null  int64  
 11   Bwd Packet Length Min        529918 non-null  int64  
 12   Bwd Packet Length Mean       529918 non-nul

In [5]:
monday.isna().any()

 Destination Port              False
 Flow Duration                 False
 Total Fwd Packets             False
 Total Backward Packets        False
Total Length of Fwd Packets    False
                               ...  
Idle Mean                      False
 Idle Std                      False
 Idle Max                      False
 Idle Min                      False
 Label                         False
Length: 79, dtype: bool

In [6]:
monday.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [7]:
x = monday.isna().any()

In [8]:
x

 Destination Port              False
 Flow Duration                 False
 Total Fwd Packets             False
 Total Backward Packets        False
Total Length of Fwd Packets    False
                               ...  
Idle Mean                      False
 Idle Std                      False
 Idle Max                      False
 Idle Min                      False
 Label                         False
Length: 79, dtype: bool

In [9]:
x.loc[x == True]

Flow Bytes/s    True
dtype: bool

In [10]:
missing_vals = monday[monday['Flow Bytes/s'].isna()]

In [11]:
missing_vals['Flow Bytes/s']

3986     NaN
18905    NaN
21063    NaN
21794    NaN
25671    NaN
          ..
472582   NaN
476509   NaN
478399   NaN
518787   NaN
524024   NaN
Name: Flow Bytes/s, Length: 64, dtype: float64

In [40]:
def missing_values(df):
    table = PrettyTable(["Column", "All Values", "Missing Values", "% of missing values"])
    NaN_columns = []
    columns = df.columns
    print(df.name)
    for column in columns:
        all_values = df[column].count()
        missing_values = df[column].isna().sum()
        if missing_values > 0:
            NaN_columns.append(column)
        percentage = f'{(missing_values / all_values)*100:.2f}'
        table.add_row([column, all_values, missing_values, f'{percentage}%'])
    data = table.get_string()
    with open(f"{df.name}_table.txt", 'w') as f:
        f.write(data)
    print(table)
    if len(NaN_columns) > 0:
        print(f"Columns with missing values: {NaN_columns}")
    print(f"Table saved as {df.name}_table.txt")

In [41]:
datasets = ['monday', 'tuesday', 'wednesday', 'thursday_morning', 'thursday_afternoon', 'friday_ddos', 'friday_pcap', 'friday_morning']
for dataset in datasets:
    if dataset == 'monday':
        missing_values(monday)
        monday = monday.dropna()
    elif dataset == 'tuesday':
        missing_values(tuesday)
        tuesday = tuesday.dropna()
    elif dataset == 'wednesday':
        missing_values(wednesday)
        wednesday = wednesday.dropna()
    elif dataset == 'thursday_morning':
        missing_values(thursday_morning)
        thursday_morning = thursday_morning.dropna()
    elif dataset == 'thursday_afternoon':
        missing_values(thursday_afternoon)
        thursday_afternoon = thursday_afternoon.dropna()
    elif dataset == 'friday_ddos':
        missing_values(friday_ddos)
        friday_ddos = friday_ddos.dropna()
    elif dataset == 'friday_pcap':
        missing_values(friday_pcap)
        friday_pcap = friday_pcap.dropna()
    elif dataset == 'friday_morning':
        missing_values(friday_morning)
        friday_morning = friday_morning.dropna()

Monday
+------------------------------+------------+----------------+---------------------+
|            Column            | All Values | Missing Values | % of missing values |
+------------------------------+------------+----------------+---------------------+
|       Destination Port       |   529918   |       0        |        0.00%        |
|         Flow Duration        |   529918   |       0        |        0.00%        |
|       Total Fwd Packets      |   529918   |       0        |        0.00%        |
|    Total Backward Packets    |   529918   |       0        |        0.00%        |
| Total Length of Fwd Packets  |   529918   |       0        |        0.00%        |
|  Total Length of Bwd Packets |   529918   |       0        |        0.00%        |
|     Fwd Packet Length Max    |   529918   |       0        |        0.00%        |
|     Fwd Packet Length Min    |   529918   |       0        |        0.00%        |
|    Fwd Packet Length Mean    |   529918   |       0     

# 3. Exploratory Data Analysis